# Set Up

In [1]:
### Import Packages ###
import os
import math as math
import pandas as pd
import random as random

### Local Packages ###
from utils.Prediction import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Main import *

# Input

In [2]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, ".."))

### DataType ###
DataTypeInput = "MONK1"

### Parameter Vector ###
ParameterVector = pd.read_csv(os.path.join(ParentDirectory, "Data", "ParameterVectors", "ParameterVector" + DataTypeInput + ".csv"))

# Local Simulation

In [9]:
SimulationConfigInput = {"DataFileInput": DataTypeInput,
                        "Seed": 44,
                        "TestProportion": 0.2,
                        "CandidateProportion": 0.8,
                        "SelectorType": "TreeEnsembleQBCFunction", 
                        "ModelType": "TreeFarmsFunction", 
                        "UniqueErrorsInput": 0,
                        "n_estimators": 100,
                        "regularization": 0.1,
                        "rashomon_bound_adder": 0.013,
                        "Type": "Classification"
                        }


In [ ]:
### Set Up ###
SimulationResults = []

### Run Simulation ###
for i in [3,4]:

    ### Set Up ###
    print("ParameterVector Simulation Number: ", i)

    ### Parameter Vector ###
    SimulationConfigInput = {"DataFileInput": ParameterVector.iloc[i]["Data"],
                            "Seed": int(ParameterVector.iloc[i]["Seed"]),
                            "TestProportion": float(ParameterVector.iloc[i]["TestProportion"]),
                            "CandidateProportion": float(ParameterVector.iloc[i]["CandidateProportion"]),
                            "SelectorType": str(ParameterVector.iloc[i]["SelectorType"]), 
                            "ModelType": str(ParameterVector.iloc[i]["ModelType"]), 
                            "UniqueErrorsInput": int(ParameterVector.iloc[i]["UniqueErrorsInput"]),
                            "n_estimators":int(ParameterVector.iloc[i]["n_estimators"]),
                            "regularization":float(ParameterVector.iloc[i]["regularization"]),
                            "rashomon_bound_adder":float(ParameterVector.iloc[i]["rashomon_bound_adder"]),
                            "Type":ParameterVector.iloc[i]["Type"]
                            }

    ### Sequential Learning Process ###
    OneIterationResults = OneIterationFunction(SimulationConfigInput)

    ### Append ###
    SimulationResults.append(OneIterationResults)

with open(os.path.join(SaveDirectory, str("MONK2/TreeFarms/Raw/0M2_DPL0.013.pkl")), 'wb') as f:
    pickle.dump(OneIterationResults, f)

ParameterVector Simulation Number:  3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iterat

In [13]:
all(SimulationResults[0]["ErrorVec"] ==1)

True